In [1]:
from bs4 import BeautifulSoup
import os
import re
import json
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import pymorphy3
directory = 'YaKlass'
os.chdir(directory)
morph = pymorphy3.MorphAnalyzer(lang='ru')

In [2]:
def read_soup(path):
    with open(path, 'r', encoding = 'utf-8') as file:
        return BeautifulSoup(file.read(), "html.parser")

def write(path, text):
    with open(path, 'w', encoding = 'utf-8') as file:
        file.write(str(text))

def term_from_sent(sent, sep = '— это '):
    left_side_sent = sent.split(sep)[0]
    right_side_sent = sent.split(sep)[1]
    term = (left_side_sent, right_side_sent)[len(left_side_sent) > len(right_side_sent)] #выделяет наименьшую по количеству слов часть предложения, которая разбивается '— это'
    #if len(left_side_sent) > len(right_side_sent):
        #term = right_side_sent
    #term = [word.lower() for word in word_tokenize(term) if word not in stops and word != '.']
    return term

def concat(terms):
    terms = [' '.join(term) for term in terms]
    for ind, term in enumerate(terms):
        if term[1] == ' ':
            terms[ind] = term[0] + term[2:]
    return terms

def text_from_soup(soup):
    '''Извлекает весь текст во всех тэгах элемента soup в строку'''
    sents_by_strip = " ".join(item.strip() for item in soup.find_all(text=True)) #находит все тэги с текстом внутри
    sents_by_strip = re.sub("\s+", " ", sents_by_strip)
    return sents_by_strip

def score(word):
    return morph.parse(word)[0].score

def merging_broken_splitted_words(words):
    '''объединяет слова, например, "Т" и "реугольник" объединятся в "Треугольник"
    работает всегда, кроме случаев, когда идут 2 и более однобуквенных слова'''
    #print('words ', words)
    if (',' in words) or ('l' in words) or ('m' in words): # сложный случай, будет обработка вручную
        return None
    
    new_words = []
    words_was_joined = False
    for ind, word in enumerate(words):
        if ind == len(words) - 1:
            if words_was_joined == False:
                new_words.append(word)
            break
        if len(word) == 1:
            new_word = words[ind] + words[ind + 1]
            if score(new_word) > score(words[ind + 1]):
                new_words.append(new_word)
                words_was_joined = True
            else:
                new_words.append(word)
        elif words_was_joined == False:
            new_words.append(word)
        else:
            words_was_joined = False
    return new_words

def term_keyword_count(sent, params = ['—', 'называть', 'называться', 'является', 'являться']):
    acceptable_values = ['—', 'называть', 'называться', 'является', 'являться']
    for param_ in params:
        if param_ not in acceptable_values:
            print('Недопустимое keyword для извлечения терминов')
            return
    count = 0
    words = [morph.parse(word)[0].normal_form for word in word_tokenize(sent)]
    for param_ in params:
        count += words.count(param_)
    return count

In [ ]:
paths = [path for path in os.listdir() if ('.html' in path) and ('new' not in path)]
for path in paths:
    with open(path, 'r', encoding = 'utf-8') as file:
        lesson = file.read()
    
    soup = BeautifulSoup(lesson, "html.parser")
        
    meta_tags = soup.find_all('meta') #Удаление всех meta-тэгов
    for meta_tag in meta_tags:
        meta_tag.decompose()
        
    reference_tags = soup.find_all(class_ = 'gxst-reference') #удаление нитжнего колонтитула
    for reference_tag in reference_tags:
        reference_tag.decompose()
    
    titles = soup.find_all(class_ = 'gxst-title') #Ставим точку в конце каждого div с классом gxst-title
    if titles != []:
        for title_tag in titles:
                
            title_text = re.sub("\s+", " ", title_tag.text)
            title_text = title_text[1:-1]
            if (title_text[:-1] != '.') or (title_text[:-1] != '?'):
                title_text += '.'
            
            title_tag.string = title_text
                
            print(title_tag) #Выводим новые значения div-ов с классом gxst-title
        
    soup = re.sub("(<!--.*?-->)", "", soup.prettify(), flags=re.DOTALL) #Удаляем комментарии
    soup = soup.replace('Ваш браузер не поддерживает HTML5 видео', 'Ваш браузер не поддерживает HTML5 видео.') #Ставим точку в конце предложения
        
    with open(path[:-5] + '_new.txt', 'w', encoding = 'utf-8') as file_2:
        file_2.write(soup)

In [ ]:
all_sents_by_strip = []
new_paths = [path for path in os.listdir() if 'new' in path]
for path in new_paths:  
    soup = read_soup(path)
    
    # Сохранение математических тэгов math в текстовый файл
    math_tags = soup.find_all('math')
    write(path.rsplit('_', maxsplit = 1)[0] + '_math.txt', math_tags)
    
    # Замена математических тэгов math на символ "M"
    for tag in soup.find_all('math'):
        tag.replace_with('m')

    # Сохранение математических формул \( \) в текстовый файл
    s = str(soup)
    matches = [match[0] for match in re.finditer(r'\\\([^\\]{0,}\\\)', s)]
    write(path.rsplit('_', maxsplit = 1)[0] + '_slash_math.txt', matches)
        
    # Замена математических формул \( \) на символ "L"
    for match in matches:
        s = s.replace(match, 'l')
    
    # Запись упрощенного результата в _simple.txt файлы
    soup = BeautifulSoup(s, "html.parser")
    write(path.rsplit('_', maxsplit = 1)[0] + '_simple.txt', soup.prettify())